In [1]:
import numpy as np 

In [2]:
from Enviar import Receber

In [3]:
import jscatter as js

In [4]:
from scipy import stats

In [5]:
import math 

In [6]:
import scipy.interpolate, scipy.optimize

In [41]:
receber=Receber(nomeRef='20-05-1-A17-f')
receber()
dados=receber.dados

In [42]:
dados[1].keys()

dict_keys(['t', 'mag', 'mag2', 'logmag2', 'energia', 'calor', 'sus', 'cumu', 'cumuE', 'tc', 'parametroOrdem', 'duracao', 'relaxacaoHistograma', 'mcsHistograma', 'A', 'B', 'numeroPontos', 'raio', 'mcsTroca', 'L', 'q', 't0'])

In [43]:
print(dados[1].get("A"))
print(dados[1].get("L"))
print(dados[1].get("q"))

1.7
10
0.021


In [44]:
concentracao=[0, 0.021, 0.025, 0.04, 0.05,  0.06, 0.07,  0.075, 0.08, 0.09,  0.1,  0.11, 0.12,0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.198, 0.2, 0.22, 0.3, 0.4]


In [45]:
class Resultado():
    def __init__(self, **kwargs):
        self.dados=kwargs.get('dados')
        
    def plotDiagrama(self, L=10, A=1.7, **kargs):
        concentracao=kargs.get("concentracao")
        p=js.grace()
        p.yaxis(label='tc',charsize=1.50)
        p.xaxis(label='q',charsize=1.50)
        x=[]
        y=[]   
        for q in concentracao: 
            for dado in self.dados:
                if dado.get('q')== q and dado.get('L')==L and dado.get('A')==A:
                    y.append(dado.get('t0'))
                    x.append(q)
        if len(y)>0:
            p.plot(x,y,symbol=-1,line=[1,1,''],legend='Q=$q')
        else:
            print("erro na seleção dos dados, seleção vazia")
    


    
    def plotDiagrama2(self,erro=0.2, L=10, A=1.7, **kargs):
        concentracao=kargs.get("concentracao")
        p=js.grace()
        p.yaxis(label='tc',charsize=1.50)
        p.xaxis(label='q',charsize=1.50)
        x=[]
        y=[]   
        for q in concentracao: 
            for dado in self.dados:
                if dado.get('q')== q and dado.get('L')==L and dado.get('A')==A:
                    if abs(dado.get('t0')-dado.get('tc'))<erro:
                        y.append(dado.get('t0'))
                        x.append(q)
        if len(y)>0:
            p.plot(x,y,symbol=-1,line=[1,1,''],legend='Q=$q')
        else:
            print("erro na seleção dos dados, seleção vazia")
    
   
    def plotDiagramaFiltrado(self,L=10, A=1.7,Z=2.5, **kargs):
        concentracao=kargs.get("concentracao")
        tamanhos=kargs.get("tamanhos")
        if not tamanhos:
            tamanhos=[L]
        p=js.grace()
        p.yaxis(label='tc',charsize=1.50)
        p.xaxis(label='q',charsize=1.50)
        for q in concentracao: 
            x=[]
            y=[]   
            for dado in self.dados:
                if dado.get('q')== q and dado.get('L')==L and dado.get('A')==A:
                    y.append(dado.get('tc'))
                    x.append(q)
            zscore=stats.zscore(np.array(y))
            X=[]
            Y=[]
            for xx,yy,zz in zip(x, y, zscore.tolist()):
                if abs(zz) < Z:
                    Y.append(yy)
                    X.append(xx)
            p.plot(X,Y,symbol=-1,line=[1,1,''],legend='Q=$q')

    def plotDiagramaMedio(self, L=10, A=1.7, Z=2.5, **kargs):
        tamanhos=kargs.get("tamanhos")
        if not tamanhos:
            tamanhos=[L]
        concentracao=kargs.get("concentracao")
        p=js.grace()
        p.yaxis(label='tc',charsize=1.50)
        p.xaxis(label='q',charsize=1.50)
        for L in tamanhos:
            xmedia=[]
            ymedia=[]
            yErro=[]
            for q in concentracao: 
                x=[]
                y=[]   
                for dado in self.dados:
                    if dado.get('q')== q and dado.get('L')==L and dado.get('A')==A:
                        y.append(dado.get('tc'))
                        x.append(q)
                zscore=stats.zscore(np.array(y))
                X=[]
                Y=[]
                for xx,yy,zz in zip(x, y, zscore.tolist()):
                    if abs(zz) < Z:
                        Y.append(yy)
                        X.append(xx)
                ymedia.append(np.average(Y))
                yErro.append(np.std(Y)/(math.sqrt(len(Y))))
                xmedia.append(np.average(X))
            expe=[2.0938997, 2.091892414662, 2.089885129325, 2.083863273312, 2.077841417299, 2.071819561285, 2.065797705272, 2.065797705272, 2.061783134597, 2.053753993246, 2.043717566558, 2.037695710545, 2.027659283857, 2.017622857168, 2.003571859804, 1.989520862441, 1.975469865077, 1.9553970117, 1.941346014336, 1.889156595557, 1.881127454206, 1.814887038063]
            p.plot(xmedia,ymedia,yErro,symbol=-1,line=[1,1,''],legend='p')
            p.plot(xmedia,expe,yErro,symbol=-1,line=[1,1,''],legend='p')
            print(xmedia)
    

    def plotDiagramaMedio2(self,erro=0.01, L=10, A=1.7, Z=2.5, **kargs):
    
        tamanhos=kargs.get("tamanhos")
        if not tamanhos:
            tamanhos=[L]
        concentracao=kargs.get("concentracao")
        p=js.grace()
        p.yaxis(label='tc',charsize=1.50)
        p.xaxis(label='q',charsize=1.50)
        for L in tamanhos:
            xmedia=[]
            ymedia=[]
            yErro=[]
            for q in concentracao: 
                x=[]
                y=[]   
                for dado in self.dados:
                    if dado.get('q')== q and dado.get('L')==L and dado.get('A')==A:
                        if abs(dado.get('tc')-dado.get('t0'))<erro:
                            y.append(dado.get('tc'))
                            x.append(q)
                zscore=stats.zscore(np.array(y))
                X=[]
                Y=[]
                for xx,yy,zz in zip(x, y, zscore.tolist()):
                    if abs(zz) < Z:
                        Y.append(yy)
                        X.append(xx)
                ymedia.append(np.average(Y))
                yErro.append(np.std(Y)/(math.sqrt(len(Y))))
                xmedia.append(np.average(X))
            print(xmedia)
            print(ymedia)
            p.plot(xmedia,ymedia,yErro,symbol=-1,line=[1,1,''],legend='p')
    def plot(self,L=10, A=1.7, x='t', y='sus', **kargs):
        concentracao=kargs.get("concentracao")
        p=js.grace()
        p.yaxis(label=y,charsize=1.50)
        p.xaxis(label=x,charsize=1.50)
        for i, q in enumerate(concentracao):
            for dado in self.dados:
                if dado.get('q')== q and dado.get('L')==L and dado.get('A')==A:
                    p.plot(dado.get(x), dado.get(y), symbol=[0,0.3,i], line =[1,1,i])

    def plotOrdem(self,L=10, A=1.7, y="parametroOrdem", **kargs):
        concentracao=kargs.get("concentracao")
        p=js.grace()
        p.yaxis(label="y",charsize=1.50)
        p.xaxis(label="x",charsize=1.50)
        for i, q in enumerate(concentracao):
            for dado in self.dados:
                if dado.get('q')== q and dado.get('L')==L and dado.get('A')==A:
                    parametroOrdemA=dado.get(y)
                    p.plot(list(range(len(parametroOrdemA))), parametroOrdemA, symbol=[0,0.3,i], line =[1,1,i])                
    
    
    def plotMedia(self,L=10, A=1.7, x='t', y='sus', **kargs):
        concentracao=kargs.get("concentracao")
        tamanhos=kargs.get("tamanhos")
        if not tamanhos:
            tamanhos=[L]
        p=js.grace()
        p.yaxis(label=y,charsize=1.50)
        p.xaxis(label=x,charsize=1.50)  
        cont=0
        for L in tamanhos:
            for i, q in enumerate(concentracao):
                somaY=np.zeros(len(dados[0].get(x)), dtype=float)
                cont=0
                for dado in self.dados:
                    if dado.get('q')== q and dado.get('L')==L and dado.get('A')==A:
                        somaY += np.array(dado.get(y))
                        if cont==0 :
                            xx=dado.get(x)
                        cont += 1
                somaY= somaY/cont
                p.plot(xx, somaY.tolist(), symbol=[0,0.3,i], line =[1,2,i] ,legend='q='+repr(q))

    def plotDiagramaCruzamentoMedia(self,L=10, A=1.7,  x='t', y='sus', **kargs):
        concentracao=kargs.get("concentracao")
        tamanhos=kargs.get("tamanhos")
        if not tamanhos:
            tamanhos=[L]
        p=js.grace()
        p.yaxis(label=y,charsize=1.50)
        p.xaxis(label=x,charsize=1.50)  
        cont=0
        tcs=[]
        for i, q in enumerate(concentracao):
            somaY1=np.zeros(len(dados[0].get(x)), dtype=float)
            somaY2=np.zeros(len(dados[0].get(x)), dtype=float)
            cont1=0
            cont2=0
            for dado in self.dados:
                if dado.get('q')== q and dado.get('L')==tamanhos[0] and dado.get('A')==A:
                    somaY1 += np.array(dado.get(y))
                    if cont==0 :
                        xx1=dado.get(x)
                    cont1 += 1
                if dado.get('p')== q and dado.get('L')==tamanhos[1] and dado.get('A')==A:
                    somaY2 += np.array(dado.get(y))
                    if cont==0 :
                        xx2=dado.get(x)
                        t0=dado.get("tc")
                    cont2 += 1
            somaY1= somaY1/cont1
            somaY2= somaY2/cont2
            tc=self.cruzamento(xx1,somaY1.tolist(),xx2, somaY2.tolist(), t0)[0]
            tcs.append(tc)
        print(tcs)
        p.plot(concentracao, tcs, symbol=[0,0.3,1], line =[1,2,1] )


    def plotFiltrados(self, Z=2.5, x='t', y='sus', L=10, A=1.7, **kargs):
        concentracao=kargs.get("concentracao")
        p=js.grace()
        p.yaxis(label=y,charsize=1.50)
        p.xaxis(label=x,charsize=1.50)
        for i, q in enumerate(concentracao):
            Y=[]
            CurvasX=[]
            CurvasY=[]
            for dado in self.dados:
                if dado.get('q')== q and dado.get('L')==L and dado.get('A')==A:
                    Y.append(dado.get('tc'))
                    CurvasX.append(dado.get(x))
                    CurvasY.append(dado.get(y))
            zscore=stats.zscore(np.array(Y))        
            for zz,tt,suss in zip(zscore.tolist(),CurvasX,CurvasY):
                if abs(zz) < Z:
                    p.plot(tt,suss, symbol=[0,0.3,i], line  =[1,1,i])#,legend='Q=$q')
    
    def PlotCruzamentos(self,A=1.7, Z=2.5, **kargs):
        concentracao=kargs.get("concentracao")
        tamanhos=kargs.get("tamanhos")
        tcmedio=[]
        tcerro=[]
        p=js.grace()
        p.yaxis(label='tc',charsize=1.50)
        p.xaxis(label='q',charsize=1.50)
        for q in concentracao:
            listasx1=[]
            listasy1=[]
            listast01=[]

            listasx2=[]
            listasy2=[]
            listast02=[]
            for dado in self.dados:
                if dado.get('q')== q and dado.get('L')==tamanhos[0] and dado.get('A')==A:
                    listasx1.append(dado.get('t'))
                    listasy1.append(dado.get('cumu'))
                    listast01.append(dado.get('tc'))
                if dado.get('q')== q and dado.get('L')==tamanhos[1] and dado.get('A')==A:
                    listasx2.append(dado.get('t'))
                    listasy2.append(dado.get('cumu'))
                    listast02.append(dado.get('tc'))
            
            listasX1=[]
            listasY1=[]
            listasT01=[]
            zscore=stats.zscore(np.array(listast01))        
            for zz,xx1,yy1, tt1 in zip(zscore.tolist(),listasx1,listasy1,listast01):
                if abs(zz) < Z:
                    listasX1.append(xx1)
                    listasY1.append(yy1)
                    listasT01.append(tt1)
            
            
            listasX2=[]
            listasY2=[]
            listasT02=[]
            zscore=stats.zscore(np.array(listast02))        
            for zz,xx2,yy2, tt2 in zip(zscore.tolist(),listasx2,listasy2,listast02):
                if abs(zz) < Z:
                    listasX2.append(xx2)
                    listasY2.append(yy2)
                    listasT02.append(tt2)

            xCrossing=[]
            yCrossing=[]
            for x1, y1  in zip(listasX1, listasY1):
                interp1 = scipy.interpolate.InterpolatedUnivariateSpline(x1, y1)
                for x2, y2 , x0 in zip(listasX2, listasY2, listasT02):
                    interp2 = scipy.interpolate.InterpolatedUnivariateSpline(x2, y2)
                    x_at_crossing = scipy.optimize.fsolve(lambda x : np.abs(interp1(x) - interp2(x)), x0=x0)
                    xc=x_at_crossing[0]
                    #yc=interp1(xc)
                    xCrossing.append(xc)
                    #yCrossing.append(yc)
            zscore=stats.zscore(np.array(xCrossing))        
            for zz,tcc in zip(zscore.tolist(),xCrossing):
                if abs(zz) < Z:
                    yCrossing.append(tcc)
            tcmedio.append(np.mean(yCrossing))
            tcerro.append(np.std(yCrossing)/math.sqrt(len(2*listasX1)))
        tcmedioN=(np.array(tcmedio)/tcmedio[0]).tolist()
        tcerroN=(np.array(tcerro)/tcmedio[0]).tolist()
        p.plot(concentracao,tcmedioN,tcerroN,symbol=-1,line=[1,1,''],legend='p')
        print(tcmedio)

    def crossings(listasX1, listasY1, listasX2, listasY2, x0): # chute inicial 
        self.xCrossing=[]
        self.yCrossing=[]
        for x1, y1  in zip(listasX1, listasY1[i]):
            interp1 = scipy.interpolate.InterpolatedUnivariateSpline(x1, y1)
            for x2, y2  in zip(listasX2, listasY2[i]):
                interp2 = scipy.interpolate.InterpolatedUnivariateSpline(x2, y2)
                x_at_crossing = scipy.optimize.fsolve(lambda x : np.abs(interp1(x) - interp2(x)), x0=x0)
                xc=x_at_crossing[0]
                yc=interp1(xc)
                self.xCrossing.append(xc)
                self.yCrossing.append(yc)
    def cruzamento(self, x1, y1, x2, y2, x0):
                interp1 = scipy.interpolate.InterpolatedUnivariateSpline(x1, y1)
                interp2 = scipy.interpolate.InterpolatedUnivariateSpline(x2, y2)
                x = scipy.optimize.fsolve(lambda x : np.abs(interp1(x) - interp2(x)), x0=x0)
                return x 
     

In [46]:
resultado = Resultado(dados=dados)

In [47]:
resultado.plotOrdem(L=10,A=1.7, concentracao=concentracao)

In [48]:
resultado.plotDiagrama(L=10, A=1.70, erro=0.05,concentracao=concentracao)

In [49]:
resultado.plotDiagramaFiltrado(L=10,A=1.70, Z=5, concentracao=concentracao)

In [51]:
resultado.plotDiagramaMedio(A=1.7,concentracao=concentracao, tamanhos=[10])

dont know how to plot this with shape (24,)
[0.0, 0.020999999999999998, 0.025, 0.039999999999999994, 0.05, 0.06000000000000001, 0.07000000000000002, 0.07499999999999998, 0.07999999999999999, 0.09, 0.1, 0.11000000000000001, 0.12000000000000002, 0.12999999999999998, 0.14000000000000004, 0.14999999999999997, 0.15999999999999998, 0.16999999999999998, 0.18, 0.198, 0.2, 0.22000000000000003, 0.29999999999999993, 0.4]


In [31]:
resultado.plotDiagramaMedio(L=10,A=1.7,concentracao=concentracao)

In [50]:
resultado.plot(L=10,A=1.7, concentracao=concentracao)

In [40]:
resultado.plotMedia(L=10,A=1.7, concentracao=concentracao)

In [18]:
resultado.plotMedia(L=10,A=1.7, concentracao=concentracao)

In [60]:
resultado.plotMedia(L=10,A=1.75,y="calor", concentracao=concentracao)

In [61]:
resultado.plotMedia(L=10,A=1.75,y="cumu", concentracao=concentracao)

In [62]:
resultado.plotMedia(A=1.75,y="cumu", concentracao=concentracao, tamanhos=[10 , 15])

Exception ignored in: <bound method GracePlot.__del__ of <jscatter.graceplot.GracePlot object at 0x7f0ec99fae48>>
Traceback (most recent call last):
  File "/home/joao/.platformio/penv/lib/python3.6/site-packages/jscatter/graceplot.py", line 1657, in __del__
    self.close()
  File "/home/joao/.platformio/penv/lib/python3.6/site-packages/jscatter/graceplot.py", line 1664, in close
    self.redraw(force=True)
  File "/home/joao/.platformio/penv/lib/python3.6/site-packages/jscatter/graceplot.py", line 2013, in redraw
    time.sleep(0.25)  # make sure on a forced redraw that the queue is flushed
KeyboardInterrupt


In [20]:
resultado.plotFiltrados(L=15,A=1.7,  concentracao=concentracao)

In [ ]:
resultado.plotMedia(A=1.7, concentracao=concentracao, tamanhos=[10 , 15, 20])

In [21]:
resultado.plotMedia(A=1.7, y='calor',concentracao=concentracao, tamanhos=[10 , 15, 20])

KeyboardInterrupt: 

In [21]:
resultado.PlotCruzamentos(A=1.7, Z=20 ,concentracao=concentracao, tamanhos=[10 , 15])

[2.0532933261333524, 2.0360111345196157, 2.031748708990801, 2.026797610672242, 2.017903097427581, 2.0113175959904708, 2.0098265895797236, 1.964819006698715, 1.95539090907056, 1.9354007231874149, 1.9351123749854386, 1.916954614858638, 1.888900889933612, 1.6175414050345873, 1.6098405025471387, 1.481702204992998, 1.4099523219588321, 1.4012498950781913, 1.3345044330901858, 1.3125832651198548, 1.2819333950219143, 1.2205435437950443, 1.0934871985570516, 1.0781870503489748, 0.9712269348824886, 0.7752201059285856, 0.48801271792609957, 0.05024018603873124]


In [ ]:
resultado.PlotCruzamentos(A=1.7, Z=20 ,concentracao=concentracao, tamanhos=[10 , 15])